# Webscraping pour obtenir le lien du dataset

In [1]:
import time

import urllib
import lxml.html
import requests
import bs4
import pandas as pd


In [2]:
url_ecb_download="https://www.ecb.europa.eu/press/key/html/downloads.en.html"
base_ecb_url="https://www.ecb.europa.eu"

In [3]:
requests.get(url_ecb_download).status_code
#Si le code retour est 200, il y a bien du contenu accessible sur la page.

200

In [4]:
#On obtient la liste de tous les liens présents sur la page

def get_all_links(url):
    '''renvoie la liste de tous les liens présents sur la page correspondant à l'url'''
    list_links=[] #on initialise avec une liste vide
    STATUS_CODE=requests.get(url).status_code
    #Si le code retour est 200, il y a bien du contenu accessible sur la page.
    if STATUS_CODE!=200:
        print("ERREUR CODE", STATUS_CODE)
    else: #il y a bien du contenu accessible sur la page
        page = urllib.request.urlopen(url_ecb_download) # on se connecte à la page pour obtenir le code source
        dom =  lxml.html.fromstring(page.read()) #permet de "lire" le code source
        for link in dom.xpath('//a/@href'): # select the url in href for all a tags(links)
            list_links.append(link)
        return list_links

In [5]:
#On obtient la liste de tous les liens présents sur la page
get_all_links(url_ecb_download)

['/home/html/index.en.html',
 '/home/html/index.en.html',
 'downloads.bg.html',
 'downloads.cs.html',
 'downloads.da.html',
 'downloads.de.html',
 'downloads.el.html',
 'downloads.en.html',
 'downloads.es.html',
 'downloads.et.html',
 'downloads.fi.html',
 'downloads.fr.html',
 'downloads.ga.html',
 'downloads.hr.html',
 'downloads.hu.html',
 'downloads.it.html',
 'downloads.lt.html',
 'downloads.lv.html',
 'downloads.mt.html',
 'downloads.nl.html',
 'downloads.pl.html',
 'downloads.pt.html',
 'downloads.ro.html',
 'downloads.sk.html',
 'downloads.sl.html',
 'downloads.sv.html',
 '/home/html/index.en.html',
 '/ecb/html/index.en.html',
 '/press/html/index.en.html',
 '/pub/html/index.en.html',
 '/stats/html/index.en.html',
 '/mopo/html/index.en.html',
 '/euro/html/index.en.html',
 '/paym/html/index.en.html',
 '/careers/html/index.en.html',
 'https://www.bankingsupervision.europa.eu/home/html/index.en.html',
 '#',
 '#',
 'javascript:window.print()',
 "javascript:window.open('https://twitt

In [6]:
def find_first_link_with_string(list_links, string):
    '''renvoie le premier lien de la liste contenant une certaine chaine de caractères'''
    for link in list_links:
        if string in link:
            return link

In [7]:
print(find_first_link_with_string(list_links=get_all_links(url_ecb_download),
                                  string="/press/key/shared/data/all_ECB_speeches.csv"))

/press/key/shared/data/all_ECB_speeches.csv?5f1650f87c081c670b20cc03e741c558


In [8]:
def get_dataset_url(url):
    '''renvoie la partie de l'url du dataset des discours de la BCE'''
    '''on utilise toutes les fonctions précédentes'''
    string="/press/key/shared/data/all_ECB_speeches.csv" #chaine de caractères identifiant le lien du dataset
    list_links=get_all_links(url) #on obtient la liste de tous les liens de la page
    url=find_first_link_with_string(list_links,string) #on cherche le lien du dataset grâce à la chaine de caractères
    return url

In [9]:
print(get_dataset_url(url_ecb_download))

/press/key/shared/data/all_ECB_speeches.csv?5f1650f87c081c670b20cc03e741c558


In [10]:
def last_update_dataset(url):
    '''renvoie la date de la dernière mise à jour du dataset'''
    STATUS_CODE=requests.get(url).status_code
    #Si le code retour est 200, il y a bien du contenu accessible sur la page.
    if STATUS_CODE!=200:
        print("ERREUR CODE", STATUS_CODE)
    else: #il y a bien du contenu accessible sur la page
        request_text = urllib.request.urlopen(url).read() #On se connecte à la page pour obtenir le code source
        page = bs4.BeautifulSoup(request_text, "lxml") 
        update_date=page.select_one("span[class*=lastUpdate]").text #on sélectionne le contenu de la classe lastUpdate
        return update_date

In [11]:
print(last_update_dataset(url=url_ecb_download))

last update: 01 January 2022


## Test de l'import du dataset

In [ ]:
start=time.time()

url="https://www.ecb.europa.eu"+ get_dataset_url(url_ecb_download)

df=pd.read_csv(url,sep = "|",encoding='utf-8')

print("Temps pris :", time.time()-start) #23 secondes avec une connexion de 30 Mbps


In [ ]:
df